# Investing On ETFs Using The Kelly Formula Part 2

## How To Allocate Capital Using The Kelly Formula

## Introduction
Suppose that we identified a trading strategy which we think could be profitable.
We implemented and backtested the trading strategy using Python programming language.
The results of the backtest is promising, the strategy's Sharpe ratio is greater than 3 with a limited maximum drawdown.
Suppose that we identified more trading strategies which are as promising as the first one.
How do we allocate our capital to these strategies?

...

## The Kelly Formula
...


## Kelly Formula In Financial Markets
In this section, we follow Thorp's [paper](https://www.researchgate.net/publication/247922818_The_Kelly_Criterion_in_Blackjack_Sports_Betting_and_the_Stock_Market) illustrating the application of Kelly criterion on optimal portfolio allocation.   

### Case 1: Single Security
The objective is to maximize the growth rate $g$ of a portfolio composed of a single security
$$ \max_{f} g(f), $$

where growth rate $g$ is
$$ g(f) = r_{Risk-free} + f m - \frac{1}{2} f^2 s^2, $$

and $f$ is the fraction of capital we allocate to the security.

Assuming that the security follows a Normal distribution with mean $m$ and variance $s^2$, the optimal allocation $f^*$ is 
$$ f^* = \frac{m}{s^2}, $$

and the optimum growth rate is 
$$ g(f^*) = r_{Risk-free} + \frac{1}{2} f^{*2} s^2. $$

### Case 2: Multiple Securities
We extend the previous case to maximize the growth rate $g$ of a portfolio composed of multiple securities
$$ \max_{F} g(F), $$

where growth rate $g$ is
$$ g(F) = r_{Risk-free} + F^T M - \frac{1}{2} F^T C F, $$

and $F$ is a vector containing fractions of capital we allocate to each security
$$ F = \left( f_1, f_2, ..., f_n \right). $$

Assuming that each security $i$ follows a Normal distribution, $M$ is the vector containing average of excess returns of each security
$$ M = \left( m_1, m_2,..., m_n \right), $$

and $C$ is the covariance matrix of the returns between securities $i$ and $j$
$$
\begin{pmatrix} 
c_{11} & c_{12} & ... & c_{1n} \\ 
c_{21} & c_{22} & ... & c_{2n} \\ 
\vdots & \vdots & \ddots & \vdots \\ 
c_{n1} & c_{n2} & ... & c_{nn} \\ 
\end{pmatrix}
$$

the optimal allocation $F^*$ is
$$ F^* = C^{-1} M, $$

and the optimum growth rate is
$$ g(F^*) = r_{Risk-free} + \frac{1}{2} F^T C F. $$

Import packages

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

Set group keys

In [2]:
groups = ['us bonds',
          'us stocks',
          'intl bonds',
          'intl stocks',
          'sectors']

Set input input files

In [3]:
input = {'us bonds': 'etf_us_bonds.pickle', 
         'us stocks': 'etf_us_stocks.pickle',
         'intl bonds': 'etf_intl_bonds.pickle',
         'intl stocks': 'etf_intl_stocks.pickle',
         'sectors': 'etf_sectors.pickle'}

Create output dictionary

In [4]:
output = {'us bonds': {},
          'us stocks': {},
          'intl bonds': {},
          'intl stocks': {},
          'sectors': {}}

Set market parameters

In [5]:
risk_free = 0.025
max_leverage = 4.00

Calculate Kelly leverage of each ETF group

In [6]:
for i in groups:    

    # Load file
    with open(input[i], 'rb') as f:
        close = pickle.load(f)
    f.close()
    
    # Daily returns for the past 6 months
    returns = close[-120:].pct_change()
    
    # Excess daily returns
    excess_returns = returns - risk_free / 250
    
    # Mean excess daily returns annualized
    M = excess_returns.mean() * 250
    
    # Covariance of daily returns
    C = returns.cov() * 250
    
    # Kelly leverage
    F = np.matmul(np.linalg.inv(C), M)
    
    # Adjust leverage
    adj_leverage = max_leverage / np.sum(np.abs(F))
    F = adj_leverage * F
    
    # Growth rate at full Kelly
    g = risk_free + np.matmul(np.transpose(F), M) - 0.5 * np.matmul(np.matmul(np.transpose(F), C), F)

    # Growth at half Kelly
    g_half = risk_free + np.matmul(np.transpose(0.5 * F), M) - 0.5 * np.matmul(np.matmul(np.transpose(0.5 * F), C), 0.5 * F)
    
    # Sharpe ratio
    sharpe = np.sqrt(np.matmul(np.matmul(np.transpose(F), C), F))
    
    # Sharpe at half Kelly
    sharpe_half = np.sqrt(np.matmul(np.matmul(np.transpose(0.5 * F), C), 0.5 * F))
    
    # Update output
    output[i]['tickers'] = list(close.columns)
    output[i]['kelly'] = list(F)
    output[i]['kelly half'] = list(0.5 * F)
    output[i]['growth'] = g
    output[i]['growth half'] = g_half
    output[i]['sharpe'] = sharpe
    output[i]['sharpe half'] = sharpe_half

### Vanguard US Bonds ETFs

The Kelly leverage for US Bonds ETFs are

In [7]:
group = 'us bonds'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly half']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
EDV,0.033435,0.016717
BIV,0.319941,0.159971
VGIT,-0.259162,-0.129581
BLV,0.128805,0.064403
VGLT,-0.059309,-0.029655
VMBS,-0.174542,-0.087271
BSV,-0.563545,-0.281772
VTIP,-0.250139,-0.125069
VGSH,-0.235515,-0.117757
BND,-0.128604,-0.064302


The growth rate and Sharpe ratio are

In [8]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth half'], output[group]['sharpe half']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.118913,0.071989
Sharpe Ratio,0.016083,0.008041


### Vanguard US Stocks ETFs

The Kelly leverage for US Stocks ETFs are

In [9]:
group = 'us stocks'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly half']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
VIG,0.041141,0.020571
VUG,1.002729,0.501365
VYM,-0.071661,-0.035830
VV,-0.316355,-0.158177
MGC,0.160262,0.080131
MGK,-0.764937,-0.382468
MGV,0.140072,0.070036
VOO,-0.523720,-0.261860
VTI,0.371481,0.185741
VTV,-0.080391,-0.040196


The growth rate and Sharpe ratio are

In [10]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth half'], output[group]['sharpe half']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.075349,0.050197
Sharpe Ratio,0.013385,0.006692


### Vanguard International Bonds ETFs

The Kelly leverage for International Bonds ETFs are

In [11]:
group = 'intl bonds'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly half']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
BNDX,2.705469,1.352734
VWOB,1.294531,0.647266


The growth rate and Sharpe ratio are

In [12]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth half'], output[group]['sharpe half']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.444818,0.235588
Sharpe Ratio,0.073694,0.036847


### Vanguard International Stocks ETFs

The Kelly leverage for International Stocks ETFs are

In [13]:
group = 'intl stocks'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly half']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
VT,0.173985,0.086992
VEU,0.827119,0.413560
VSS,-0.098323,-0.049162
VEA,0.691189,0.345595
VGK,-0.146990,-0.073495
VPL,-0.764251,-0.382125
VNQI,0.390855,0.195428
VXUS,-0.878502,-0.439251
VWO,0.028785,0.014393


The growth rate and Sharpe ratio are

In [14]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth half'], output[group]['sharpe half']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.126155,0.075764
Sharpe Ratio,0.038642,0.019321


### Vanguard Sectors ETFs

The Kelly leverage for sectors ETFs are

In [16]:
group = 'sectors'

pd.DataFrame({'Kelly Leverage': output[group]['kelly'],
              'Half-Kelly Leverage': output[group]['kelly half']},
             index=output[group]['tickers'])

,Kelly Leverage,Half-Kelly Leverage
VOX,0.050629,0.025314
VCR,-0.346022,-0.173011
VDC,-0.104041,-0.052020
VDE,-0.488407,-0.244203
VFH,0.714675,0.357338
VHT,-0.816149,-0.408074
VIS,0.159580,0.079790
VGT,0.616926,0.308463
VAW,0.016476,0.008238
VNQ,0.514397,0.257199


The growth rate and Sharpe ratio are

In [17]:
pd.DataFrame({'Kelly Leverage': [output[group]['growth'], output[group]['sharpe']],
              'Half-Kelly Leverage': [output[group]['growth half'], output[group]['sharpe half']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Kelly Leverage,Half-Kelly Leverage
Growth Rate,0.533054,0.281720
Sharpe Ratio,0.146787,0.073393


## Conclusion
...